# The Ultimate Guide To AB Testing

## AB testing helps us understand and quantify change 

If we have a product or service that works in a certain way, we often want to make changes to improve this. When we make a change we want to be confident that the impact is "real" i.e. not caused by random chance. In order to be able to measure this change we can use AB testing.

In an AB test we split the users into different groups; the control group stays with the same experience or product and then the other groups each receive a variation of this (often there is only one other variant). We then measure key metrics for each group and compare the results. AB testing can be used in many fields but is often used in product development.

To make this more concrete for the rest of the article - imagine we are running a website and are looking to improve the sales conversion - i.e the proportion of users signing up who purchase on our website. We want to run an AB test to see if changing the styling of a key product page increases the conversion. The users are randomly split into two groups, we show the control group the old website and we show the users in the variant group the modified styling. We run the test for a period of time and observe how many from each group go on to purchase - we then calculate the statistics to determine whether the conversion rate has improved for the new style.

You can use AB testing for other metrics such as average spend but the analysis is a little different.

When calculating the statistics for an AB test there are two main schools of thought - `Freqentist` and `Bayesian`

## Some notation

Before we get into the detail let's define the number of users entering the test as $n$ and the number of users who convert as $s$ (I chose s for success).

We can then split users these between the two variants. We use $n_C$ and $n_V$ to represent the number of users in the control and variant respectively. We use $s_C$ and $s_V$ to represent the number of users who convert in the control and variant respectively. Note $n = n_C + n_V$ and $s = s_C + s_V$.

Now we can write the conversion rate $p$ as $p = \frac{s}{n}$ and for control $p_C = \frac{s_C}{n_C}$ and for the variant $p_V = \frac{s_V}{n_V}$.

## Frequentist Vs Bayesian thinking

`Freqentist` and `Bayesian` approaches to testing differ in how they calculate test statistics but also in how they think about the problem. It is worth describing these two different ways of thinking about testing before going into the calculations.

### Frequentist

`Frequentists` view the `true` theoretical conversion rate as a fixed unique value - for instance in our example it could be 10%. This underlying value is also called the `population conversion rate` but it can't be directly observed as we can't realistically have everyone in the world try out our website. Instead we observe the `sample conversion rate` from the sample of users we do observe. This may be slightly different from the underlying conversion rate due to the randomness in the sample.

In this `Frequentist` setting we use known properties about samples to work out if the variant sample is `significantly` different from the control. 

### Bayesian

In a `Bayesian` setting instead of considering the `population conversion rate` as a single unobservable value it is considered as a distribution. For instance it could be a distribution centered around 10% as shown below. This probability distribution expresses the idea that we think the conversion rate is itself random but most likely to be around 10%.

In [16]:
import numpy as np
import plotly.graph_objs as go
from plotly.offline import iplot
from scipy.stats import distributions

def bayesian_example_distribution():
    figure = go.FigureWidget()
    
    # Beta values to plot
    n = 200
    step = 1 / n
    x = np.arange(0, 1, step)
    beta = distributions.beta(a=10, b=100)
    y = beta.pdf(x)

    figure.add_scatter(x=x, y=y, fill='tozeroy', opacity=0.5)
    
    # labels
    figure.layout.title = 'Example Conversion Rate Distribution'
    figure.layout.xaxis.title = 'Conversion rate'
    figure.layout.yaxis.title = 'Probability density'
    figure.layout.xaxis.tickformat = '%'
    figure.layout.yaxis.showticklabels = False

    return figure

example_distribution = bayesian_example_distribution()

In [18]:
example_distribution

FigureWidget({
    'data': [{'fill': 'tozeroy',
              'opacity': 0.5,
              'type': 'scatter',…

## Calculating Test Significance - the idea

Now we understand how `Freqentist` and `Bayesian` frameworks view conversion rates we can explain how the test statistics are calculated and what they mean.

### Frequentist

#### Summary

In a Freqentist setting we use hypothesis testing - we assume there is no change in conversion between control and the variant groups then observe the results. We calculate the probability of observing a difference in conversion rates at least as large as we see (the p value) and if very unlikely we conclude the assumption was wrong and that the conversion rates must be different.

#### Hypothesis testing

In a frequentist setting we initially assume that both the control and variant groups have the same underlying conversion rate (we assume that they are sampled from the same population). This assumption or hypothesis is known as the `null hypothesis` denoted $H_0$.

The `alternative hypothesis`, $H_1$ is just the opposite of the null hypothesis - that the underlying conversion rates are different.$^1$

As we initially assume the underlying conversion rates are the same we expect the conversion rates of the control sample and variant sample to be close to each other and the underlying true value.

Say if the underlying conversion rate was 10% (remember in practice we don't know this) then we would expect both the control and the variant to have roughly 10% of users convert under the null hypothesis.

If the variant sample has an average conversion rate `significantly` different to the control we reject the null hypothesis and accept the alternative hypothesis.

In order to decide whether the difference is significant we work out the $p$ value. The $p$ value is the probability of seeing a conversion rate in the variant at least as far away from the control conversion rate as we observed. If this $p$ is smaller than the preset significance value (also known as $\alpha$ - often this is 5%) then we reject the null hypothesis. The $p$ value has corresponding `critical values` which are the conversion rates required to reject the null hypothesis (either much larger or much smaller than the control value)

When sampling from a distribution where each user has a fixed probability of converting (the underlying conversion rate) then the conversion rate of a sample of users with size $n$ will tend to a normal distribution as we increase $n$. This is due to the central limit theorem and allows us to calculate the $p$ value.

---
*footnote*

$^1$: Strictly speaking this formulation is the two tail variant - for a one tail hypothesis test the alternate hypothesis asserts that the variant conversion rate is greater than the control conversion rate or even vice-versa. However the one tail test is less common.

#### A detailed look at the p value

The Bernoulli distribution is a discrete distribution having two possible outcomes (success and failure). The shorthand $X \sim {\rm Bernoulli}(p)$ is used to indicate that the random variable $X$ has the Bernoulli distribution with parameter $p$ - the chance of success, where $0 < p < 1$. We can write the probability mass function as

$$
\begin{aligned}
P(X=0) &= 1-p \\
P(X=1) &= p
\end{aligned}
$$

We can model each website user as a Bernoulli random variable where they have the value 1 if they convert and 0 if they do not. Therefore we assume the control and the variant groups are both samples of random Bernoulli variables. Note that the conversion rate of each sample is the same as the mean of the random variables - both are equal to the proportion of success in the sample.

In general if we have a distribution we can imagine taking many samples and then calculating the mean (or conversion rate) of each sample. These means would then have a distribution around the true mean - we call this distribution the sampling distribution of the mean. The central limit theorem tells us that the sampling distribution tends towards a normal distribution with a certain mean $\mu$ and variance $\sigma^2$.

The mean can be estimated by the mean of the sample (i.e. the observed conversion rate). The central limit theorem tells us that the standard deviation $\sigma_{\bar{x}}$ of the sampling distribution of the mean (also know as the standard error) is equal to the population standard deviation $\sigma$ divided by the square root of the size of the sample $n$.
$$
\sigma_{\bar{x}} = \frac{\sigma}{\sqrt{n}}
$$

We don't know the population standard deviation but we can estimate it using the sample standard deviation. So we assume that the control sample conversion rate $\mu(C,n_C)\sim\mathcal{N}(\mu_C,\sigma_C^2)$ where 

$$
\mu_C=c_C\quad \sigma_C = \frac{\text{s}_C}{\sqrt{n_C}}
$$

Where $\text{s}_C$ is the standard deviation of the control sample. Similarly we assume the variant conversion rate $\mu(V,n_V)\sim\mathcal{N}(\mu_V,\sigma_V^2)$ where 

$$
\mu_V=c_V\quad \sigma_V = \frac{\text{s}_V}{\sqrt{n_V}}
$$
Where $\text{s}_V$ is the standard deviation of the variant sample.

As both samples are made up of Bernoulli random variables the standard deviations are defined as

$$\text{s}_C = \sqrt{c_C(1-c_C)}\\\text{s}_V = \sqrt{c_V(1-c_V)}$$



We are interested in $D = \mu(V,n_V) - \mu(C,n_C)$ the difference between the two conversion rates. As we have assumed they are normally distributed we can say that the difference is also normally distributed. The Freqentist statistics all come from approximating the difference distribution as normal. 

$$
D = \mu(V,n_V) - \mu(C,n_C) \sim\mathcal{N}(\mu_V - \mu_C, \sigma_C^2+\sigma_V^2) = \mathcal{N}(\mu_V -\mu_C, \frac{\text{s}_C^2}{n_C} + \frac{\text{s}_V^2}{n_V})
$$
Note the variance of the difference distribution is equal to the sum of the variances:
$$
\sigma_D = \sigma_C^2+\sigma_V^2 = \frac{\text{s}_C^2}{n_C}+\frac{\text{s}_V^2}{n_V}
$$

Under the null hypothesis $H_0$ we assume $\mu_C=\mu_V$ hence

$$
\mu(V,n_V) - \mu(C,n_C) \sim\mathcal{N}(0,\frac{\text{s}_C^2}{n_C}+\frac{\text{s}_V^2}{n_V})
$$


Specifically we want to know the p value, the probability that difference was at least as extreme as we observed.
I.e. 

$$
P(|\mu(V,n_V) - \mu(C,n_C)| > |c_V - c_C|) = 2 * P(Z > \frac{|\mu_V - \mu_C|}{\sqrt{\frac{\text{s}_C^2}{n_C}+\frac{\text{s}_V^2}{n_V}}}   )
$$

Where Z is the standard normal.

#### Visualising frequentist statistical significance with an example

As an example if we assume, the true underlying conversion rate is 20% and the control and variant groups both had 100 users. If we observed 21 conversions in control and 25 conversions the variant we may wonder whether this difference of 4% conversion is a significant result. 

Below we calculate the conversion rates and the variance of the control sampling distribution and the variant sampling distribution. We also approximate the variance of the difference distribution by summing the variances of the control and variant groups.

Next we plot the difference distribution assuming $H_0$ (centered about 0 because the conversion rates are assume to be equal) and the difference distribution assuming $H_1$ (centered about the observed difference). The result is significant if the observed difference (the mean of the difference distribution under $H_1$) is to the right of the right critical value or to the left of the left critical value.

In the example below the result is not significant.

In [1]:
true_conversion_rate = 0.2

control_users = 10000
control_user_conversions = 2000

variant_users = 10000
variant_user_conversions = 2100

control_conversion_rate = control_user_conversions / control_users
variant_conversion_rate = variant_user_conversions / variant_users
control_variance = (control_conversion_rate * (1 - control_conversion_rate)) 
variant_variance = (variant_conversion_rate * (1 - variant_conversion_rate))
diffence_variance = (control_variance / control_users) + (variant_variance / variant_users)
observed_difference = variant_conversion_rate - control_conversion_rate

In [21]:
frequentist_difference(observed_difference=observed_difference,
                       diffence_variance=diffence_variance,
                       alpha=0.1,
                       two_tale=True,
                       show_alpha=True,
                       show_power=True)

FigureWidget({
    'data': [{'name': 'Null hypothesis difference distribution',
              'type': 'scatter…

#### Type I and Type II error

In Frequentist testing a `type I` error is the rejection of a true null hypothesis (also known as a "false positive" finding or conclusion). This is equivalent to concluding the variant is significantly different when it really isn't - it just had a higher/lower conversion by chance. The type I error is controlled by the $\alpha$ value. If the $\alpha$ is set at 5% then in the long run we would expect a type I error 5% of the time. The type I error corresponds to the sample mean falling in the green tails in the plot above.

A `type II` error is when we do not reject a false null hypothesis (also known as a "false negative"). This is equivalent to not concluding the variant is better even though it is. Type II error is sometimes denoted as $\beta$. The power is defined as one minus the chance of type II error so ideally we want a large power value. The power value is related to the number of samples, the alpha value, the current conversion rate and the expected uplift (also know as the minimal detectable effect). Type II error is normally controlled by preselecting an expected uplift, calculating the required number of samples using power analysis and then running the test until there are enough samples.

It is a very bad idea to calculate the p value before this point because repeated significance calculations can increase type I error. We will come back to this later in simulations.

#### Deriving the minimum sample size for Frequentist test

It is best to first to visualise the power on a chart to then derive the relationship between sample size, minimal detectable effect, baseline conversion rate, alpha and power. The minimal detectable effect is the smallest uplift or difference you expect and want to measure.

We want the variant conversion rate to be greater than the control conversion rate by the minimal detectable effect.

The power is the probability of finding this minimal increase. As above we can plot the difference distributions assuming the mean difference is zero, i.e. under the null hypothesis and also assuming the mean difference is the minimal detectable effect, i.e. under the alternative hypothesis. On the chart the power is the probability under the alternative hypothesis difference distribution to the right of the critical value (the 95th percentile of the null difference distribution).

In [ ]:
frequentist_difference(observed_difference=0.1,
                       diffence_variance=diffence_variance,
                       alpha=0.05,
                       two_tale=True,
                       show_alpha=True,
                       show_power=True)

Mathematically we can write the following (where $d$ is the minimal detectable effect and $Z$ is the standard normal):

$$
\begin{aligned}
\text{power} &= P(D > \sigma_D * Z_{\frac{\alpha}{2}}|D\sim\mathcal{N}(d, \sigma_D))\\
\text{power} &= P\left(\frac{D - d}{\sigma_D} > \frac{\sigma_D * Z_{\frac{\alpha}{2}} - d}{\sigma_D}|
    D\sim\mathcal{N}(d, \sigma_D)\right)\\
\text{power} &= P\left(Z > Z_{\frac{\alpha}{2}} - \frac{d}{\sigma_D}\right)
\end{aligned}
$$

So we can define 
$$Z_{\text{power}} = Z_{\frac{\alpha}{2}} - \frac{d}{\sigma_D}$$ 

But as

$$\beta = 1 - \text{power}$$

It follows 

$$
\begin{aligned}
Z_{\beta} & = -Z_{\text{power}}\\
Z_{\beta}  & = \frac{d}{\sigma_D} - Z_{\frac{\alpha}{2}}
\end{aligned}
$$

Rearranging

$$
\frac{d}{\sigma_D} = Z_{\frac{\alpha}{2}} + Z_{\beta}
$$

Expanding $\sigma_D$
$$
\frac{d}{\sqrt{\frac{s_C^2}{n_C} + \frac{s_V^2}{n_V}}} = Z_{\frac{\alpha}{2}} + Z_{\beta}
$$
Assuming $n_C = n_V = n$ and rearranging for n we see
$$
n = \sqrt{\frac{\left(Z_{\frac{\alpha}{2}} + Z_{\beta}\right)^2\left(s_C^2+s_V^2\right)}{d^2}}
$$


### Bayesian

$$ P(A|B) = \frac{P(B|A)P(A)}{P(B)} $$

- describe the prior 
- describe the update process
- plot the prior and the posterior
- calculate the probability of beating control
- explain the loss as a concept

## What can possibly go wrong ... ?

- repeated testing

## Code for ab-testing

In [5]:
import numpy as np
from scipy.stats import norm
from scipy.special import betaln

In [195]:
# Input to test results

# conversions
control_users = 10000
control_user_conversions = 2000
variant_users = 10000
variant_user_conversions = 2100

# settings
alpha = 0.05
beta = 0.2
power = 1 - beta
# bayesian prior settings
prior_alpha = 1
prior_beta = 1


In [196]:
# calulations

# Frequentist
control_conversion_rate = control_user_conversions / control_users
variant_conversion_rate = variant_user_conversions / variant_users
observed_difference = variant_conversion_rate - control_conversion_rate
control_variance = control_conversion_rate * (1 - control_conversion_rate)
variant_variance = variant_conversion_rate * (1 - variant_conversion_rate)
p_hat = (control_user_conversions + variant_user_conversions) / (control_users + variant_users)
pooled_variance = (p_hat * (1 - p_hat)) * ((1 / control_users) + (1 / variant_users))
# very similar in practise:
# satterthwaite_variance = (control_variance / control_users) + (variant_variance / variant_users)
pooled_standard_deviation = pooled_variance ** 0.5

# Assuming two tail test (hence the * 2 on p_value)
z_value = observed_difference / pooled_standard_deviation
p_value = norm.sf(abs(z_value)) * 2
frequentist_significant = p_value < alpha



# Bayesian
control_posterior_alpha = prior_alpha + control_user_conversions
control_posterior_beta = prior_beta + control_users - control_user_conversions
variant_posterior_alpha = prior_alpha + variant_user_conversions
variant_posterior_beta = prior_beta + variant_users - variant_user_conversions
n_beta_samples = 1_000_000
diff_distribution = (
    np.random.beta(variant_posterior_alpha, variant_posterior_beta, n_beta_samples) - 
    np.random.beta(control_posterior_alpha, control_posterior_beta, n_beta_samples))
chance_to_beat_control = (diff_distribution > 0).sum() / diff_distribution.shape[0]
# # Could calculate directly using the following line but then harder to inspect quantiles
# chance_to_beat_control = prob_beat_control_faster(
#     control_posterior_alpha, control_posterior_beta,
#     variant_posterior_alpha, variant_posterior_beta)
# Assuming Two tail - not really a Bayesian concept
bayesian_significant = chance_to_beat_control < (alpha / 2) or chance_to_beat_control > (1 - (alpha / 2))

In [197]:
p_value

0.07984985065721206

In [198]:
chance_to_beat_control

0.960067

In [199]:
frequentist_significant

False

In [200]:
bayesian_significant

False

In [124]:
# Useful ab test functions
def prob_beat_control_sample(alpha_c, beta_c, alpha_v, beta_v,
                             n=10_000, random_state=101):
    """Calculate chance to beat control using sampling

    Speed:
        n=10_000 => 2.43 ms
        n=100_000 => 21.8 ms
        n=1_000_000 => 226 ms
    """
    np.random.seed(random_state)
    diff_distribution = (np.random.beta(alpha_v, beta_v, n) -
                         np.random.beta(alpha_c, beta_c, n))
    return (diff_distribution > 0).sum() / diff_distribution.shape[0]


def prob_beat_control_faster(alpha_c, beta_c, alpha_v, beta_v):
    """Calculate chance to beat control using sampling

    Speed:
        min(alpha_v, beta_v) = 1_000 => 192 µs
        min(alpha_v, beta_v) = 10_000 => 1.22 ms
        min(alpha_v, beta_v) = 100_000 => 11.3 ms
    """
    if alpha_v < beta_v:
        i_s = np.arange(alpha_v)
        prob_beat_control = np.exp(
            betaln(alpha_c + i_s, beta_v + beta_c) -
            np.log(beta_v + i_s) -
            betaln(1 + i_s, beta_v) -
            betaln(alpha_c, beta_c)).sum()
    else:
        i_s = np.arange(beta_v)
        prob_beat_control = 1 - np.exp(
            betaln(beta_c + i_s, alpha_c + alpha_v) -
            np.log(alpha_v + i_s) -
            betaln(1 + i_s, alpha_v) -
            betaln(alpha_c, beta_c)).sum()
    return np.clip(prob_beat_control, 0, 1)


### plotting

In [6]:
import numpy as np


from scipy.stats import norm
from ipywidgets import widgets

In [1]:
def frequentist_difference(observed_difference,
                           diffence_variance,
                           alpha=0.05,
                           two_tale=True,
                           show_alpha=True,
                           show_power=False):
    null_difference_dist = norm(loc=0, scale=diffence_variance ** 0.5)
    alt_difference_dist = norm(loc=observed_difference, scale=diffence_variance ** 0.5)

    start_prob = 0.0001
    end_prob = 1 - start_prob
    null_start, null_end  = null_difference_dist.ppf([start_prob, end_prob])
    alt_start, alt_end = alt_difference_dist.ppf([start_prob, end_prob])
    start = min(null_start, alt_start)
    end = max(null_end, alt_end)
    
    n_points = 500
    step = (end - start) / n_points
    plot_conversion_rates = np.arange(start, end, step)
    null_plot_conversion_rates_probs = null_difference_dist.pdf(plot_conversion_rates)
    alt_plot_conversion_rates_probs = alt_difference_dist.pdf(plot_conversion_rates)

    fig = go.FigureWidget()
    fig.add_scatter(
        x=plot_conversion_rates,
        y=null_plot_conversion_rates_probs,
        name="Null hypothesis difference distribution")
#     fig.add_scatter(
#         x=plot_conversion_rates,
#         y=alt_plot_conversion_rates_probs,
#         name="Alternative hypothesis difference distribution")
    fig.layout.title = 'Difference distributions'
    fig.layout.xaxis.title = 'Difference d'
    fig.layout.yaxis.title = 'Probability density'

    if two_tale:
            alpha = alpha / 2
            
    if show_alpha:
        plot_alpha_right_x = np.arange(null_difference_dist.ppf(1 - alpha), end, step)
        plot_alpha_right_y = null_difference_dist.pdf(plot_alpha_right_x)
        plot_alpha_left_x = np.arange(start, null_difference_dist.ppf(alpha), step)
        plot_alpha_left_y = null_difference_dist.pdf(plot_alpha_left_x)
        right = fig.add_scatter(
            x=plot_alpha_right_x,
            y=plot_alpha_right_y,
            line=dict(color="green"),
            fill='tozeroy',
            name=f"{alpha:.2%} prob")
        left = fig.add_scatter(
            x=plot_alpha_left_x,
            y=plot_alpha_left_y,
            fill='tozeroy',
            line=dict(color="green"),
            name=f"{alpha:.2%} prob",
            visible=two_tale)

    if show_power:
        plot_power_right_x = np.arange(null_difference_dist.ppf(1 - alpha), end, step)
        plot_power_right_y = alt_difference_dist.pdf(plot_power_right_x)
        power = 1 - alt_difference_dist.cdf(null_difference_dist.ppf(1 - alpha))
        power_line = fig.add_scatter(
            x=plot_power_right_x,
            y=plot_power_right_y,
            line=dict(color="orange"),
            fill='tozeroy',
            name=f"Power: {power:.2%}")
    return fig

In [2]:
from plotly.subplots import make_subplots

In [3]:
figure = go.FigureWidget()
    
# Beta values to plot
n = 200
step = 1 / n
x = np.arange(0, 1, step)


for a, b in [(2,8), (20,80),(200, 800),(2000, 8000)]:
    beta = distributions.beta(a=a, b=b)
    y = beta.pdf(x)
    figure.add_scatter(x=x, y=y, fill='tozeroy',opacity=0.5,
                      name=f'Belief after seeing {a+b} users and {a} conversions ')
    

# labels
figure.layout.title = 'Example Conversion Rate Distribution'
figure.layout.xaxis.title = 'Conversion rate'
figure.layout.yaxis.title = 'Probability density'
figure.layout.xaxis.tickformat = '%'
figure.layout.yaxis.showticklabels = False
figure

NameError: name 'go' is not defined